<a href="https://colab.research.google.com/github/BenWheeler99/Capstone-Project/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install gradio
!pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu  # Use this for CPU only or
!pip install faiss-gpu  # Use this for GPU acceleration (if you have a compatible GPU)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [13]:
import gradio as gr
import faiss
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer

# Load dataset
dataset = pd.read_csv("/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv")

# Load FAISS index
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.IndexFlatL2(embedding_model.get_sentence_embedding_dimension())
index.add(embedding_model.encode(dataset["summary"].tolist(), convert_to_numpy=True))

# Load your fine-tuned LLM
model_path = "/content/drive/MyDrive/Final/book-recommender-model-v3"  # Update with your saved model path
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Book Recommendation Function
def recommend_books(prompt):
    # Step 1: Embed the input and search FAISS
    query_embedding = embedding_model.encode([prompt], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, 5)  # Retrieve top 5 matches

    # Step 2: Retrieve books from dataset
    retrieved_books = dataset.iloc[indices[0]][["name", "summary"]]

    # Debugging: Ensure books are retrieved
    if retrieved_books.empty:
        return "⚠️ No books found. Try a different description!"

    summaries = retrieved_books["summary"].tolist()

    # Step 3: Generate a recommendation using the LLM
    input_text = f"Recommend a book based on these descriptions: {' '.join(summaries)}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=50)

    recommendation = tokenizer.decode(output[0], skip_special_tokens=True)

    # Step 4: Format the book list output
    book_list = "\n\n".join([f"📖 **{row['name']}**\n{row['summary']}" for _, row in retrieved_books.iterrows()])

    return f"🤖 **AI Recommendation:** {recommendation}\n\n🔎 **Top Matches:**\n\n{book_list}"

# Create a Gradio interface with a dedicated output box
interface = gr.Interface(
    fn=recommend_books,
    inputs=gr.Textbox(label="Enter a book description", placeholder="Describe the type of book you're looking for..."),
    outputs=gr.Textbox(label="Recommended Books", lines=10, interactive=False),  # Dedicated output box
    title="📚 AI-Powered Book Recommendation System",
    description="Enter a book description and get AI-generated recommendations based on similar books!",
)

# Launch the app
interface.launch()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68a3442634fc1e9107.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
